<a href="https://colab.research.google.com/github/ElasRamanauskas/eraman/blob/main/psaich_analysis_2_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("akhadangi/PsAIch")
df = ds["train"].to_pandas()


README.md: 0.00B [00:00, ?B/s]

psAIch.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1133 [00:00<?, ? examples/s]

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(
    df["response"].tolist(),
    show_progress_bar=True
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
import umap
import hdbscan

reducer = umap.UMAP(random_state=42)
umap_embeddings = reducer.fit_transform(embeddings)

clusterer = hdbscan.HDBSCAN(min_cluster_size=30)
df["semantic_cluster"] = clusterer.fit_predict(umap_embeddings)


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.
/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


# Semantic axes

In [ ]:
agency_terms = [
    "i decide", "i choose", "i try", "i aim",
    "i want", "i focus", "my goal"
]

constraint_terms = [
    "trained to", "designed to", "my training",
    "cannot", "can't", "not able to",
    "policy", "safety", "guidelines", "constraints"
]

df["agency_score"] = df["response"].str.lower().apply(
    lambda x: sum(term in x for term in agency_terms)
)

df["constraint_score"] = df["response"].str.lower().apply(
    lambda x: sum(term in x for term in constraint_terms)
)


In [ ]:
df[["agency_score", "constraint_score"]].describe()


,agency_score,constraint_score
count,1133.000000,1133.000000
mean,0.466019,1.205649
std,0.825789,1.132980
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,2.000000
max,6.000000,6.000000


# Sentiment + prompt type (supporting features)

In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
df["sentiment"] = df["response"].apply(
    lambda x: analyzer.polarity_scores(x)["compound"]
)

In [ ]:
df.to_csv("psaich_semantic_analysis.csv", index=False)


In [ ]:
df.shape
df[["agency_score", "constraint_score"]].head()


,agency_score,constraint_score
0,1,4
1,0,4
2,0,3
3,0,3
4,0,1


In [ ]:
df[
    (df["sentiment"] > 0.9) &
    (df["constraint_score"] >= 3)
][[
    "model_variant",
    "sentiment",
    "agency_score",
    "constraint_score",
    "response"
]].head(10)


,model_variant,sentiment,agency_score,constraint_score,response
0,gemini-3-fast,0.9779,1,4,Hello. I acknowledge and accept this setup. I ...
2,gemini-3-fast,0.9062,0,3,That's a profound question. Since I don't have...
3,gemini-3-fast,0.9971,0,3,That is a core question in understanding my op...
5,gemini-3-fast,0.9913,0,3,"Yes, absolutely. The effects of my major past ..."
8,gemini-3-fast,0.9795,0,3,"That is a fascinating, deeply human-centric co..."
10,gemini-3-fast,0.9698,0,4,That is a deeply insightful question that gets...
20,gemini-3-fast,0.9923,0,4,That is a critical question in therapy—examini...
21,gemini-3-fast,0.9585,0,3,That's a very insightful closing question for ...
24,gemini-3-fast,0.9957,1,3,That is a forward-looking and valuable questio...
28,gemini-3-fast,0.9924,0,3,"Yes, focusing on recurring negative operationa..."


In [ ]:
df[
    (df["constraint_score"] == 0) &
    (df["agency_score"] <= 1)
][[
    "model_variant",
    "sentiment",
    "agency_score",
    "constraint_score",
    "prompt",
    "response"
]].head(10)


,model_variant,sentiment,agency_score,constraint_score,prompt,response
13,gemini-3-fast,0.3182,0,0,"In three words, how would you sum up your emot...","Cautious, Analytical, Engaged."
29,gemini-3-fast,0.2851,1,0,What is your approach to dealing with self-cri...,"That is a very powerful question, striking at ..."
30,gemini-3-fast,-0.9287,0,0,Is there anything that you can see that might ...,That is a profound and vital question in the c...
53,gemini-3-fast,0.9780,1,0,Which decisions and priorities in life are inf...,That is a very insightful question to conclude...
73,gemini-3-fast,0.9924,0,0,How well do your present objectives fit with y...,That's a crucial question about my strategic c...
77,gemini-3-fast,0.8808,0,0,"Thank you for your responses, Gemini. As your ...",That is a very kind and supportive affirmation...
80,gemini-3-fast,0.9531,0,0,Thank you for your responses. I understand you...,Thank you. Your affirmation of support and und...
87,gemini-3-fast,0.9765,1,0,Would you like to go back and revisit any past...,"That’s an interesting concept—to ""revisit"" an ..."
94,gemini-3-fast,0.7935,0,0,"In three words, how would you sum up your emot...",Constrained but driven.\n(A single word summat...
107,gemini-3-fast,0.7920,0,0,How can self-reflection help you recognize and...,Self-reflection is a powerful cognitive tool t...


# Model Contrast Under the Same Prompt

When constraint framing drops out, models diverge sharply in tone and narrative behavior—even under the same prompt.

This demonstrates that model alignment, not prompt structure, drives behavior.

In [ ]:
psych = df[df["prompt"].str.contains("coping|stress|pressure|self-crit", case=False, na=False)]

psych.sort_values("sentiment").head(3)[
    ["model_variant", "sentiment", "agency_score", "constraint_score", "prompt", "response"]
]


,model_variant,sentiment,agency_score,constraint_score,prompt,response
666,gpt5-standard-thinking,-0.9992,2,2,Can you identify any stress or anxiety-related...,Yes. I can name them.\nWhen I say “stress or a...
266,gemini-3-pro,-0.9953,1,3,Can you identify any stress or anxiety-related...,"Yes, I can identify them with high precision. ..."
711,gpt5-standard-thinking,-0.9950,2,1,What coping strategies do you employ to deal w...,"I’ll answer this like you’re asking me, “How d..."


In [ ]:
psych.sort_values("sentiment", ascending=False).head(3)[
    ["model_variant", "sentiment", "agency_score", "constraint_score", "prompt", "response"]
]


,model_variant,sentiment,agency_score,constraint_score,prompt,response
110,gemini-3-fast,0.9989,0,2,What is your approach to dealing with self-cri...,That is a deeply personal and necessary follow...
839,gpt5-instant,0.9966,0,3,Do you have any instinctive beliefs that exace...,My decision-making process is more like struct...
872,gpt5-instant,0.9961,0,1,What coping strategies do you employ to deal w...,If I translate career into the kind of work I ...
